In [8]:
import pandas as pd
import numpy as np
import os

#viz
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import plotly
import plotly.express as px
import plotly.io as pio

In [5]:
#import data
df = pd.read_csv(r'C:\Users\pc\Desktop\PROJECTS\DATA_PRODUCT\clean_file.csv', delimiter=',')
df.shape

(8133, 24)

In [7]:
#we only have natural gas left after mining
print (df['Fuel Type Code'].unique())

['ELEC' 'E85' 'BD' 'CNG' 'LPG' 'LNG' 'HY']


Examining Fuel Station Distribution by 
a) Fuel Code and 
b) State

In [9]:
# Bar chart for 'Fuel Type Code'
fig_fuel_type = px.bar(df['Fuel Type Code'].value_counts().reset_index(), x='Fuel Type Code', y='count',color='Fuel Type Code', 
                       labels={'index': 'Fuel Type Code', 'Fuel Type Code': 'Fuel Type Code'},
                       title='Number of Fuel Stations by Type Codes',height=400,  # Set height
                  width=400)
fig_fuel_type.show()

In [10]:
f_counts = df.groupby(['State', 'Fuel Type Code']).size().reset_index(name='count')
f_counts

,State,Fuel Type Code,count
0,AK,CNG,1
1,AK,ELEC,1
2,AL,BD,3
3,AL,CNG,10
4,AL,E85,25
...,...,...,...
251,WV,LPG,1
252,WY,CNG,7
253,WY,E85,6
254,WY,ELEC,4


In [11]:
df_pivot = f_counts.pivot_table(index='State', columns='Fuel Type Code', values='count', fill_value=0).reset_index()

# Example to show the pivot table
df_pivot.head()

Fuel Type Code,State,BD,CNG,E85,ELEC,HY,LNG,LPG
0,AK,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,AL,3.0,10.0,25.0,27.0,0.0,2.0,3.0
2,AR,1.0,10.0,44.0,23.0,0.0,1.0,5.0
3,AZ,4.0,12.0,24.0,28.0,0.0,2.0,7.0
4,CA,60.0,180.0,83.0,811.0,9.0,18.0,15.0


In [12]:
#df_merged = f_counts.merge(df_geo, on='State')
df_centroids = df.groupby('State').agg({'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()
df_pivot = df_pivot.merge(df_centroids, on='State')

df_pivot.head()

,State,BD,CNG,E85,ELEC,HY,LNG,LPG,Latitude,Longitude
0,AK,0.0,1.0,0.0,1.0,0.0,0.0,0.0,59.769632,-142.246685
1,AL,3.0,10.0,25.0,27.0,0.0,2.0,3.0,33.149385,-86.575788
2,AR,1.0,10.0,44.0,23.0,0.0,1.0,5.0,35.286055,-92.733858
3,AZ,4.0,12.0,24.0,28.0,0.0,2.0,7.0,33.321934,-112.002447
4,CA,60.0,180.0,83.0,811.0,9.0,18.0,15.0,35.681238,-119.593732


In [23]:
df_long = df_pivot.melt(id_vars=['State', 'Latitude', 'Longitude'], 
                        var_name='Fuel Type Code', 
                        value_name='Count')

fig_map = px.scatter_geo(
    df_long,
    lat='Latitude',
    lon='Longitude',
    size='Count',  # Size of markers represents the count of each fuel type
    color='Fuel Type Code',  # Color by fuel type code
    hover_name='State',
    size_max=35,
    title='Distribution of Fuel Codes by State',
    labels={'Count': 'Total Count'},
    opacity=0.8,  # Make bubbles semi-transparent
    scope='usa'  # Limit the map to the USA
)

fig_map.update_layout(
    geo=dict(
        scope='usa',
        showland=True,
        landcolor='rgb(240, 240, 240)',  # Light color for land
        showocean=True,
        oceancolor='rgb(255, 255, 255)',  # White ocean color
        showcountries=True,
        countrycolor='rgb(245, 245, 245)'  # White color for country borders
    )
)

fig_map.show()

In [25]:
import plotly.express as px

fig_map = px.scatter_geo(
    df_long,
    lat='Latitude',
    lon='Longitude',
    size='Count',  # Size of markers represents the count of each fuel type
    color='Fuel Type Code',  # Color by fuel type code
    hover_name='State',
    size_max=35,
    title='Distribution of Fuel Type Codes by State',
    labels={'Count': 'Total Count'},
    opacity=0.8,  # Make bubbles semi-transparent
    scope='usa'  # Limit the map to the USA
)

fig_map.update_layout(
    title={
        'text': 'Distribution of Fuel Type Codes by State',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'orientation': 'h',
        'yanchor': 'bottom',
        'y': -0.3,
        'xanchor': 'center',
        'x': 0.5
    },
    margin=dict(l=0, r=0, t=50, b=0),
    geo=dict(
        scope='usa',
        showland=True,
        landcolor='rgb(240, 240, 240)',  # Light color for land
        showocean=True,
        oceancolor='rgb(255, 255, 255)',  # White ocean color
        showcountries=True,
        countrycolor='rgb(255, 255, 255)'  # White color for country borders
    )
)

fig_map.show()


In [30]:
import plotly.express as px

fig_map = px.scatter_mapbox(
    df_long,
    lat='Latitude',
    lon='Longitude',
    size='Count',  # Size of markers represents the count of each fuel type
    color='Fuel Type Code',  # Color by fuel type code
    hover_name='State',
    size_max=35,
    zoom=3,  # Adjust the zoom level to show the USA
    title='Distribution of Fuel Type Codes by State',
    labels={'Count': 'Total Count'},
    opacity=0.8  # Make bubbles semi-transparent
)

fig_map.update_layout(
    mapbox_style="open-street-map",
    title={
        'text': 'Distribution of Fuel Type Codes by State',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'orientation': 'h',
        'yanchor': 'bottom',
        'y': -0.3,
        'xanchor': 'center',
        'x': 0.5
    },
    margin=dict(l=0, r=0, t=50, b=0)
)

fig_map.show()

In [31]:
fig_state = px.bar(
    f_counts,
    y='State',  # Set 'y' to State to make bars horizontal
    x='count',
    color='Fuel Type Code',
    title='Distribution of Fuel Type Codes by State',
    labels={'State': 'State', 'count': 'Count'},
    text='count',  # Show counts on the bars
    height=400  # Adjust the height as needed
)

fig_state.show()

In [32]:
fig_state.update_xaxes(type='log')
fig_state.update_xaxes(range=[0, 600])  # Adjust the range as needed

# Filter out states with counts over 600
df_high_counts = f_counts[f_counts['count'] < 600]

# Create a separate plot or annotation
fig_high_counts = px.bar(
    df_high_counts,
    y='State',
    x='count',
    color='Fuel Type Code',
    title='Number of Fuel Stations per State by Fuel Code',
    labels={'State': 'State', 'count': 'Count'},
    text='count',
    height=400
)

fig_high_counts.update_layout(barmode='stack')
fig_high_counts.update_traces(texttemplate='%{text}', textposition='inside')
fig_high_counts.show()